In [128]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


In [129]:
def replace_non_numbers(item):
    try:
        return int(item)
    except ValueError:
        return np.nan

def eval_list_str(x):

    # Remove square brackets and newline characters
    x = x.strip('[]\n')
    x = x.replace('...', '')
    x = x.replace("''", '')
    x = x.replace("'", '')
    x = x.replace(",", '')

    # Split the string by spaces
    number_strings = x.split()

    # Convert number strings to integers
    number_list = [replace_non_numbers(num) for num in number_strings]
    return number_list


In [130]:
data = pd.read_csv("data_joined.csv", index_col=0)

# Baseline para classificar atividade muscular

* 0: sem atividade muscular
* 1: com atividade muscular

In [131]:
data["std"] = data["emg"].apply(lambda x: np.std(pd.Series(eval_list_str(x)).dropna()))
data["mean"] = data["emg"].apply(lambda x: np.mean(pd.Series(eval_list_str(x)).dropna()))
data["min"] = data["emg"].apply(lambda x: np.min(pd.Series(eval_list_str(x)).dropna()))
data["max"] = data["emg"].apply(lambda x: np.max(pd.Series(eval_list_str(x)).dropna()))
data = data.fillna(method="ffill")

C:\Users\lucas\AppData\Local\Temp\ipykernel_15264\551365725.py:5: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [132]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [133]:
X = data[["std", "mean", "min", "max"]].iloc[:32]
y = data["label"].iloc[:32]

X_oos = data[["std", "mean", "min", "max"]].iloc[32:]
y_oos = data["label"].iloc[32:]


In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state=32)

In [135]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_oos = scaler.transform(X_oos)

In [136]:
def PlotConfusionMatrix(y, x, titulo:str, isRidge:bool):
    if isRidge:
        cm = confusion_matrix(y, ridge_cf.predict(x))
    else:
        cm = confusion_matrix(y, rf_cf.predict(x))

    fig = px.imshow(cm,
                    labels=dict(x="Predicted", y="True"),
                    x=['Classe 0', 'Classe 1'],
                    y=['Classe 0', 'Classe 1'],
                    color_continuous_scale='Blues')

    for i in range(len(cm)):
        for j in range(len(cm[i])):
            fig.add_annotation(
                x=j, y=i,
                text=str(cm[i][j]),
                showarrow=False,
                font=dict(color="orange", size=15),
                align="center"
            )

    fig.update_layout(title=titulo,
                    xaxis_title="Classe Predita",
                    yaxis_title="Classe Verdadeira")
    fig.update_traces(showscale=True, colorbar=dict(title="Counts"))

    fig.update_layout(font=dict(size=12))

    fig.show()

### Ridge Classifier

In [137]:
ridge_cf = RidgeClassifier()
ridge_cf.fit(X_train, y_train)
print("Train Score: ", ridge_cf.score(X_train, y_train))
print("Test Score: ", ridge_cf.score(X_test, y_test))
print("OOS Score: ", ridge_cf.score(X_oos, y_oos))

Train Score:  1.0
Test Score:  0.875
OOS Score:  0.95


#### Report Test (rc)

In [138]:
PlotConfusionMatrix(y_test, X_test, "Matriz de Confusão Ridge Classifier", 1)

In [139]:
class_report = classification_report(y_test, ridge_cf.predict(X_test), target_names=['Class 0', 'Class 1'])
print(class_report)


              precision    recall  f1-score   support

     Class 0       0.75      1.00      0.86         3
     Class 1       1.00      0.80      0.89         5

    accuracy                           0.88         8
   macro avg       0.88      0.90      0.87         8
weighted avg       0.91      0.88      0.88         8



#### Report OOS (rc)

In [140]:
PlotConfusionMatrix(y_oos, X_oos, "Matriz de Confusão Ridge Classifier", 1)

In [141]:
class_report = classification_report(y_oos, ridge_cf.predict(X_oos), target_names=['Class 0', 'Class 1'])
print(class_report)


              precision    recall  f1-score   support

     Class 0       0.95      0.95      0.95        20
     Class 1       0.95      0.95      0.95        20

    accuracy                           0.95        40
   macro avg       0.95      0.95      0.95        40
weighted avg       0.95      0.95      0.95        40



### Random Forest

In [142]:
rf_cf = RandomForestClassifier()
rf_cf.fit(X_train, y_train)
print("Train Score: ", rf_cf.score(X_train, y_train))
print("Test Score: ", rf_cf.score(X_test, y_test))
print("OOs Score: ",  rf_cf.score(X_oos, y_oos))

Train Score:  1.0
Test Score:  1.0
OOs Score:  1.0


#### Report Test (RF)

In [143]:
PlotConfusionMatrix(y_test, X_test, "Matriz de Confusão Random Forest", 0)

In [144]:
class_report = classification_report(y_test, rf_cf.predict(X_test), target_names=['Class 0', 'Class 1'])
print(class_report)


              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00         3
     Class 1       1.00      1.00      1.00         5

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



#### Report OOS (RF)

In [145]:
PlotConfusionMatrix(y_oos, X_oos, "Matriz de Confusão Random Forest", 0)

In [146]:
class_report = classification_report(y_oos, rf_cf.predict(X_oos), target_names=['Class 0', 'Class 1'])
print(class_report)


              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00        20
     Class 1       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

